<a href="https://colab.research.google.com/github/Rohit-78958/Qdrant/blob/master/qdrant_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U sentence-transformers

In [ ]:
pip install -U qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4

In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
documents = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

In [ ]:
client = QdrantClient(":memory:")

In [ ]:
client.create_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [ ]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

In [ ]:
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("alien invasion").tolist(),
    limit=1,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'The War of the Worlds', 'description': 'A Martian invasion of Earth throws humanity into chaos.', 'author': 'H.G. Wells', 'year': 1898} score: 0.5700933395837129


In [ ]:
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("alien invasion").tolist(),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="year", range=models.Range(gte=2000))]
    ),
    limit=2,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'The Three-Body Problem', 'description': 'Humans encounter an alien civilization that lives in a dying system.', 'author': 'Liu Cixin', 'year': 2008} score: 0.4590293090137795
{'name': 'The Hunger Games', 'description': 'A dystopian society where teenagers are forced to fight to the death in a televised spectacle.', 'author': 'Suzanne Collins', 'year': 2008} score: 0.16074508691053407


**.............Neural Search Engine.....................**

In [ ]:
!wget https://storage.googleapis.com/generall-shared-data/startups_demo.json

--2024-11-07 04:56:24--  https://storage.googleapis.com/generall-shared-data/startups_demo.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22205751 (21M) [application/json]
Saving to: ‘startups_demo.json’

startups_demo.json  100%[===================>]  21.18M  23.3MB/s    in 0.9s    

2024-11-07 04:56:25 (23.3 MB/s) - ‘startups_demo.json’ saved [22205751/22205751]



In [ ]:
pip install sentence-transformers numpy pandas tqdm


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2"
)

In [ ]:
df = pd.read_json("./startups_demo.json", lines=True)

In [ ]:
vectors = model.encode(
    [row.alt + ". " + row.description for row in df.itertuples()],
    show_progress_bar=True,
)

Batches:   0%|          | 0/1265 [00:00<?, ?it/s]

In [ ]:
vectors.shape
# > (40474, 384)

(40474, 384)

In [ ]:
np.save("startup_vectors.npy", vectors, allow_pickle=False)

In [ ]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance



In [ ]:
if not client.collection_exists("startups"):
    client.create_collection(
        collection_name="startups",
        vectors_config=VectorParams(size=384, distance=Distance.COSINE),
    )

In [ ]:
fd = open("./startups_demo.json")

# payload is now an iterator over startup data
payload = map(json.loads, fd)

# Load all vectors into memory, numpy array works as iterable for itself.
# Other option would be to use Mmap, if you don't want to load all data into RAM
vectors = np.load("./startup_vectors.npy")

In [ ]:
client.upload_collection(
    collection_name="startups",
    vectors=vectors,
    payload=payload,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256,  # How many vectors will be uploaded in a single request?
)

In [ ]:
hits = client.query_points(
    collection_name="startups",
    query=encoder.encode("fashion").tolist(),
    query_filter=None
    limit=3,
).points

#hits

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'Stylesight', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/39623-ebef07b8fadfed7813ea596025cb4bce-thumb_jpg.jpg?buster=1408289603', 'alt': 'Stylesight -  fashion', 'description': '', 'link': 'http://www.stylesight.com/', 'city': 'New York'} score: 0.7719707483194118
{'name': 'Shoptiques', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/79468-058a052c27ed13baf8e509dcd1178880-thumb_jpg.jpg?buster=1407426708', 'alt': 'Shoptiques -  fashion', 'description': '', 'link': 'http://www.shoptiques.com', 'city': 'New York'} score: 0.7548952769362056
{'name': 'Fashion85', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/349642-8543683c8155c15bb6b152efcb96c34c-thumb_jpg.jpg?buster=1393298465', 'alt': 'Fashion85 -  mobile fashion', 'description': 'Fashion Crowd-Opinion App\nEvery day, millions of women have several questions of style, either about the combination of looks, makeup, accessories or even questions related to purchasing products. Sometimes i

In [ ]:
#must filter technique

from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()


        # Use `vector` for search for closest vectors in the collection
        search_result = client.query_points(
            collection_name=self.collection_name,
            query=vector,
            query_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="city",
            match=models.MatchValue(value="Chicago")
        )
    ]
),
# If you don't want any filters for now
            limit=5,  # 5 the most closest results is enough
        ).points
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        # payloads = [hit.payload for hit in search_result]
        # return payloads
        return search_result

In [ ]:
#must not filter technique

from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()


        # Use `vector` for search for closest vectors in the collection
        search_result = client.query_points(
            collection_name=self.collection_name,
            query=vector,
            query_filter = models.Filter(
    must_not=[
        models.FieldCondition(
            key="city",
            match=models.MatchValue(value="Chicago")
        )
    ]
),
# If you don't want any filters for now
            limit=5,  # 5 the most closest results is enough
        ).points
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        # payloads = [hit.payload for hit in search_result]
        # return payloads
        return search_result

In [ ]:
#should filter technique

from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()


        # Use `vector` for search for closest vectors in the collection
        search_result = client.query_points(
            collection_name=self.collection_name,
            query=vector,
            query_filter = models.Filter(
    should=[
        models.FieldCondition(
            key="city",
            match=models.MatchValue(value="Chicago")
        ),
        models.FieldCondition(
            key="city",
            match=models.MatchValue(value="New York")
        )
    ]
)
,
# If you don't want any filters for now
            limit=5,  # 5 the most closest results is enough
        ).points
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        # payloads = [hit.payload for hit in search_result]
        # return payloads
        return search_result

In [ ]:
# Create a neural searcher instance
neural_searcher = NeuralSearcher(collection_name="startups")
search_results = neural_searcher.search(text="pandora style fashion in new yoirk")

# search_results
for hit in search_results:
    print(hit.payload, "score:", hit.score)

{'name': 'Gekks', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/597007-2cbe730f50e417d66e92fa339ff5ae7f-thumb_jpg.jpg?buster=1422897483', 'alt': 'Gekks -  retail fashion Designer footwear', 'description': 'Innovative no-show sock\nGekks are thin, breathable liner socks that grip to the inside of your boat shoes, loafers, drivers, and more. Our customers get to keep their sockless style by slipping in and out of their shoes barefoot, yet add comfort and anti-stench properties while inside ...', 'link': 'http://www.mygekks.com', 'city': 'Chicago'} score: 0.40567598066328314
{'name': 'MaBaker', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/706547-18cac9fa2ec094d004c07bb41eaa6a9e-thumb_jpg.jpg?buster=1432234428', 'alt': 'MaBaker -  mobile social media platforms art online dating', 'description': 'Your tattoo can chat to another\nNow your Tattoo can like and be liked by other Tattoos around it! TatChat’s aim is to attract and unite all people that bare body art, 

**FILTERINGS**

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00
